# Updating Hosted Feature Layer using Excel Table

#### Ensure that input table (Excel) and target layer (hosted layer) share the same field names
#### The hosted layer is imported from ArcGIS Online.

Reference: https://developers.arcgis.com/python/samples/updating-features-in-a-feature-layer/

In [249]:
from pathlib import Path
import sys

from arcgis.gis import GIS, Item
from arcgis.features import FeatureLayerCollection
from arcgis.mapping import WebMap
from datetime import datetime
import arcpy
import os
import pandas as pd
import numpy as np

In [169]:
from arcgis.features.managers import FeatureLayerManager
from arcgis import features
from arcgis import geometry

In [3]:
#aprx = arcpy.mp.ArcGISProject('current')

In [170]:
gis = GIS("https://mmsp.maps.arcgis.com", "MatsuzakiEiji", "daimoku@9ichiban*")

In [230]:
fsItemId = "cb692b9ff5ef40fba0fce9ebc4d590f7" # viaduct associated feature layer (do not choose scene layer)
featureLayer = source.content.get(fsItemId)
target_layer = featureLayer.layers[0]

In [231]:
#querying without any conditions returns all the features
target_fset = target_layer.query()
target_fset.sdf.head()

,OBJECTID,uniqueID,Status1,ID,PierNumber,nPierNumber,Layer,Type,end_date,CP,created_user,created_date,last_edited_user,last_edited_date,GlobalID,planned_date,SHAPE
0,1,1,4,N01-STR-YN-10320,MT01-01,MT011A,N-01_PILE,1,2023-03-28,N-01,NSCREXUSER,2023-11-10 06:09:02,NSCREXUSER,2023-11-10 06:09:02,322d34e8-6fc9-4c48-bdb6-5159ada095d0,2023-04-14,"{""rings"": [[[13445235.8498, 1676714.3137], [13..."
1,2,2,4,N01-STR-YN-10320,MT01-01,MT011B,N-01_PILE,1,2023-03-27,N-01,NSCREXUSER,2023-11-10 06:09:02,NSCREXUSER,2023-11-10 06:09:02,62327712-ed75-4ab9-b8c4-9a59be0f3899,2023-04-14,"{""rings"": [[[13445232.7179, 1676717.7801], [13..."


In [173]:
input_table = r"C:\Users\oc3512\Dropbox\01-Railway\02-NSCR-Ex\01-N2\03-During-Construction\02-Civil\03-Viaduct\01-Masterlist\02-Compiled\Viaduct_MasterList.xlsx"
input_table = pd.read_excel(input_table)
input_table.head()

,Layer,PierNumber,Type,Status1,CP,idd,uniqueID,nPierNumber,end_date,ID,planned_date,updated
0,N-01_PILE,MT01-01,1,4,N-01,486.0,1,MT011A,2023-03-28,N01-STR-YN-10320,2023-04-14,2024-01-10
1,N-01_PILE,MT01-01,1,4,N-01,487.0,2,MT011B,2023-03-27,N01-STR-YN-10320,2023-04-14,2024-01-10
2,N-01_PILE,MT01-01,1,4,N-01,488.0,3,MT011C,2023-03-30,N01-STR-YN-10320,2023-04-14,2024-01-10
3,N-01_PILE,MT01-01,1,4,N-01,489.0,4,MT011D,2023-03-31,N01-STR-YN-10320,2023-04-14,2024-01-10
4,N-01_PILE,MT01-01,1,4,N-01,490.0,5,MT011E,2023-04-01,N01-STR-YN-10320,2023-04-14,2024-01-10


In [232]:
# First get all features
all_target_features = target_fset.features
all_target_features[0]

{"geometry": {"rings": [[[13445235.8498, 1676714.3137], [13445235.6773, 1676714.2939], [13445235.5045, 1676714.3133], [13445235.3405, 1676714.3709], [13445235.1933, 1676714.4638], [13445235.0702, 1676714.5873], [13445234.9778, 1676714.7352], [13445234.9201, 1676714.9003], [13445234.9005, 1676715.0739], [13445234.9198, 1676715.2477], [13445234.977, 1676715.4129], [13445235.0692, 1676715.561], [13445235.192, 1676715.6847], [13445235.339, 1676715.7779], [13445235.503, 1676715.8358], [13445235.6756, 1676715.8555], [13445235.8483, 1676715.8361], [13445236.0123, 1676715.7786], [13445236.1595, 1676715.6857], [13445236.2826, 1676715.5622], [13445236.3751, 1676715.4143], [13445236.4328, 1676715.2492], [13445236.4523, 1676715.0755], [13445236.433, 1676714.9018], [13445236.3759, 1676714.7367], [13445236.2836, 1676714.5885], [13445236.1608, 1676714.4648], [13445236.0138, 1676714.3716], [13445235.8498, 1676714.3137]]], "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"OBJEC

In [175]:
from copy import deepcopy

In [255]:
for_update = []
uniqueIDs = [f for f in input_table['uniqueID']]

for unique_id in uniqueIDs:
    # Status for target
    original_feature = [f for f in all_target_features if f.attributes['uniqueID'] == unique_id][0]
    to_be_updated = deepcopy(original_feature)
    
    # get the matching row from input_table
    #matching_uniqueid = input_table.where(input_table.uniqueID == unique_id).dropna()
    
    # get the status for input and target layers
    ## Input 
    id = input_table.index[input_table['uniqueID'] == unique_id]
    status_input = input_table['Status1'].loc[id].iloc[0]
    enddate_input = pd.to_datetime(input_table['end_date'].loc[id].iloc[0]).date()
    print('Input: {}'.format(status_input))
    print('Input: {}'.format(enddate_input))
    
    ## Target
    status_target = original_feature.attributes['Status1']
    objectid_target = original_feature.attributes['OBJECTID']
    
    timestamp = original_feature.attributes['end_date'] / 1000
    enddate_target = datetime.fromtimestamp(timestamp).date()
    
    ## If input and target does not match in status and end_date, append to for_update list
    if status_input != status_target or enddate_input != enddate_target:
        to_be_updated = {
            'attributes': {
                'OBJECTID': objectid_target,
                'Status1' : status_input,
                'end_date': enddate_input
            }
        }
        for_update.append(to_be_updated)

Input: 4
Input: 2023-03-28
Input: 4
Input: 2023-03-27


In [259]:
if len(for_update) == 0:
    print('No need to update target feature layer')
else:
    target_layer.edit_features(updates=for_update)


No need to update target feature layer
